## Programming without indices (axpy)

### Preliminaries

<p><font color=red> Again, copy this notebook so that you don't corrupt the original!  Then you can "play" with the copy of the notebook all you want! </font> </p>

<p>In this notebook, we show how the FLAME notation (the notation in which vectors and/or matrices are partitioned into regions) can be leveraged to implement linear algebra operations without using indices (which are the root of all evil in programming...).</p>

Let's start by creating vectors $ x = \left( \begin{array}{r} 1 \\ 2 \\ 3 \end{array} \right) $, $ y = \left( \begin{array}{r} -1.0 \\ 0.0 \\ -2.0 \end{array} \right) $ and a scalar $ \alpha = 2.5 $.  

Execute the code in the box by clicking in the box and then on "Cell -> Run".  Alternatively, click on the box and push "Shift" and "Return" (or "Enter") together.

In [26]:
include("../laff/laff.jl")
using .laff

# create two column vectors x and y.  
x = [1, 2, 3]
@show x

y = [-1.0, 0.0, -2.0]
@show y

α = 2.5
@show α;

yold = fill(0.0, 3)
laff.copy!( y, yold )

x = [1, 2, 3]
y = 

[-1.0, 0.0, -2.0]
α = 2.5


### Axpy as a simple routine

<p>
Here is a simple routine for computing $ y := \alpha x + y $:
</p>

In [27]:
function axpy!( α, x, y )

    m = length(x)
    
    for i in 1:m
        y[ i ] = α * x[ i ] + y[ i ]
    end
end

axpy! (generic function with 1 method)

Be sure the run the above box, or this notebook won't know about the routine!!!

Now, execute

In [28]:
laff.copy!( yold, y )

println( "y before axpy:")
println( y )

axpy!( α, x, y)

println( "y after axpy: " )
println( y )

println( "compare new y to alpha * x + yold:" )
println( y - ( α * x + yold ) )

y before axpy:
[-1.0, 0.0, -2.0]
y after axpy: 
[1.5, 5.0, 5.5]
compare new y to alpha * x + yold:
[0.0, 0.0, 0.0]


### An implementation with the FLAMEPy Application Programming Interface (API)

We now show how to implement this same routine using the FLAMEPy API.

Start by visiting the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage </a>.  Follow along with the video and paste the resulting code below.  Then follow along with the video and add the appropriate commands.

Here is the algorithm as presented in Unit 1.6.5.  
<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/FLAME_Algorithm_Axpy.png" alt="some_text">

In the video for Unit 1.6.6, you are encouraged to us the Spark webpage and the above algorithm to implement the axpy operation.

<b> Note: </b>
<ul>
<li> Make sure that the name of the routine is <code> axpy_unb </code>.
<li> Make sure you order the parameters <code> alpha, x, y </code>.
<li> Make sure you work through the vectors in the right direction.
</ul>

In [49]:
include("../laff/laff.jl")
include("../flame.jl")
using .laff
using .flame

function axpy_unb!(alpha, x, y)

    xT, xB  = flame.part_2x1(x, 0, "TOP")

    yT, yB  = flame.part_2x1(y, 0, "TOP")

    while size(xT)[1] < size(x)[1]

        x0, chi1, x2    = flame.repart_2x1_to_3x1(xT, xB, 1, "BOTTOM")

        y0, psi1, y2    = flame.repart_2x1_to_3x1(yT, yB, 1, "BOTTOM")

        #------------------------------------------------------------#

        psi1 = alpha * chi1 + psi1
        
        #------------------------------------------------------------#

        xT, xB  = flame.cont_with_3x1_to_2x1(x0, chi1, x2, "TOP")

        yT, yB  = flame.cont_with_3x1_to_2x1(y0, psi1, y2, "TOP")
    end
        
    flame.merge_2x1!(yT, yB, y)

end



axpy_unb! (generic function with 1 method)

In [50]:
include("../laff/laff.jl")
using .laff

laff.copy!( yold, y )

α = -2.0 

axpy_unb!( α, x, y )

println( "y after axpy: " )
println( y )

println( "compare new y to alpha * x + yold:" )
println( y - ( α * x + yold ) )

y after axpy: 
[-3.0, -4.0, -8.0]
compare new y to alpha * x + yold:
[0.0, 0.0, 0.0]


The output should be:
<code>
y after axpy: 
[-3.0, -4.0, -8.0]
compare new y to alpha * x + yold:
[0.0, 0.0, 0.0]
</code>